# Librairies

In [1]:
import project_functions as pf
import pandas as pd

# Portfolio Analysis


Importons les données :

In [2]:
# récupérons les données des prix sauvegardés dans le fichier prices.csv
prices = pd.read_csv('datas/prices.csv', sep=',')
prices = prices.set_index('Date')

## Analyse des returns

Nous ne prenons uniquement les assets avec des returns anualisés positifs.

In [3]:
# Calculons les rendements journaliers
daily_returns = pf.daily_asset_return(prices)

# Calculons les rendements annuels
annualy_returns = pf.annualy_asset_return(daily_returns)

# On ne concerne uniquement les index avec rendements annuels positifs
positive_returns = annualy_returns[annualy_returns > 0]

# On récupère les tickers des index avec rendements positifs
tickers = positive_returns.index.to_list()

Actualisons le dataframe prices avec uniquement les tickers avec un rendement positif

In [4]:
prices = prices[tickers]

## Critères ESG

Nous allons créer notre portefeuille d'actions en se basant sur les critères ESG. Comme vu en cours cela s'appelle un ESG filter :
* exclude x% of firms with the worts ESG score
* keep firms with the best ESG momentum
* take a specific KPI

## Analyse des Volatilités

Nous calculons la volatilites des actions avec la fonction asset_vol.

In [5]:
# Calculons la volatilité annuelle de chaque actif et trions les valeurs par ordre décroissant
total_vol = pf.asset_vol(prices).sort_values(ascending=False)
total_vol

ASML.AS     2881.585328
ASM.AS      1511.832169
GLPG.AS      799.040403
HOLCO.AS     730.578229
DSM.AS       710.757258
HAL.AS       524.112975
WKL.AS       445.673250
HEIA.AS      364.857634
HYDRA.AS     358.546970
AKZA.AS      343.236776
BESI.AS      333.216240
HEIO.AS      330.274225
NEDAP.AS     240.468437
TWEKA.AS     208.263251
AALB.AS      189.898765
RAND.AS      186.146327
AMG.AS       181.959332
CRBN.AS      156.809416
ARCAD.AS     138.583458
REN.AS       114.713417
AD.AS        107.982181
KENDR.AS     104.703251
VPK.AS        99.915111
BRILL.AS      97.163977
SLIGR.AS      95.395512
ACOMO.AS      91.739262
SHELL.AS      67.304495
VLK.AS        67.076211
AJAX.AS       62.672310
BGHL.AS       62.296180
SNOW.AS       54.752081
VASTN.AS      52.496398
BRNL.AS       44.420426
HEIJM.AS      41.899145
INGA.AS       37.973304
TOM2.AS       33.287206
SBMO.AS       32.508720
VTA.AS        23.193275
PORF.AS       21.044427
VALUE.AS      18.694897
IEX.AS        17.617157
CTAC.AS       16

## Enregistrement des tickers

Nous devons enregistrer les tickers sélectionnés dans le fichier tickers.txt

In [6]:
pf.tickers_storage(tickers, 'datas/tickers.txt')